<a href="https://colab.research.google.com/github/fwangliberty/AIoTDesign-Frontend/blob/master/ensembel3_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensembling pre-trained DNN, CNN2D, Random Forest and DecisionTree 


We will load the pre-trained DNN, CNN2D, Random Forest and Decision Tree to classify network anormalies in CICIDS2017 dataset. More specifically, the test dataset with connection-based features and smote is used for random forest and decision tree. For DNN model, we use test dataset with connection-based and service-type features and smote.

## Preparing the data
First, import dependencies.

In [1]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import os
from os.path import join
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt


### Define Metrics

In [2]:
def display_metrics(y_test, y_pred, label_names):
  print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

  print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
  print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
  print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

  print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
  print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
  print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

  print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

  print('\nClassification Report\n')
  print(classification_report(y_test, y_pred, target_names=label_names))

In [3]:
def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d

In [4]:
# chganges label from string to integer/index
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

# 1. Locating CSV files

In [5]:
# All columns
col_names = np.array(['dst sport count', 'src dport count', 'dst src count', 'dport count', 'sport count', 'dst host count','src host count','Source Port', 'Destination Port',
                      'Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
                      'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
                      'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                      'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
                      'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
                      'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                      'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
                      'Avg Bwd Segment Size','Subflow Fwd Packets', 'Subflow Fwd Bytes',
                      'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
                      'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                      'Idle Std', 'Idle Max', 'Idle Min', 'Label'])

### Option 1. Connect to Google Drive 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Only load the test dataset

In [7]:
test_path = '/content/drive/My Drive/CICIDS2017/test_set_ext78_2.csv'

### Option 2. Connect to Local Machine

In [ ]:
test_path = '../data/cicids2017clean/test_set_ext78_2.csv'

# 2. Loading CSV Dataset

In [8]:
df_test = pd.read_csv(test_path, names=col_names, skiprows=1)  
print('Test set size: ', df_test.shape)

Test set size:  (188483, 79)


In [ ]:
df_test.describe()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,188483.000000,...,188483.000000,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05,1.884830e+05
mean,9.625308,37.861701,49.864025,48.026984,10.434315,55.607434,57.249322,42825.772430,6398.028565,8.699867,...,3.475565,-4.185286e+02,8.921930e+04,3.533351e+04,1.448189e+05,6.831514e+04,1.412079e+07,8.001563e+05,1.473072e+07,1.352238e+07
std,25.759133,40.995800,44.175627,39.217990,25.916640,41.132627,40.537490,19694.807197,16123.799361,4.737968,...,355.424245,1.932191e+05,6.619472e+05,3.716482e+05,9.742883e+05,5.992893e+05,3.123964e+07,5.987927e+06,3.206597e+07,3.101737e+07
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,-8.388531e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000,1.000000,4.000000,2.000000,1.000000,8.000000,11.000000,35960.000000,53.000000,6.000000,...,0.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000,16.000000,37.000000,48.000000,1.000000,60.000000,64.000000,50181.000000,80.000000,6.000000,...,1.000000,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.000000,95.000000,99.000000,96.000000,2.000000,99.000000,99.000000,57348.000000,443.000000,6.000000,...,2.000000,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.281430e+06,0.000000e+00,5.282144e+06,5.158305e+06
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,65536.000000,65536.000000,17.000000,...,152621.000000,1.380000e+02,1.070000e+08,4.230000e+07,1.070000e+08,1.070000e+08,1.200000e+08,7.260000e+07,1.200000e+08,1.200000e+08


## For Decision tree and random forest models, we use unnormalized data set

In [9]:
df_test_origin = df_test.copy()

# 3. Adding Is_Http Feature

In [10]:
http_port = [80, 8080, 443]

In [11]:
df_test.insert(1, "is_http", "")

In [12]:
for i, row in df_test.iterrows():
    if row['Destination Port'] in http_port:
        df_test.at[i, 'is_http'] = 1
    else:
        df_test.at[i, 'is_http'] =  0

# 4. Encoding Datasets

### Encoding test dataset

In [13]:
df_label = df_test['Label']
data = df_test.drop(columns=['Label'])
Xtest = data.values
y_test = encode_label(df_label.values)

In [14]:
X_test = Xtest.copy()

In [15]:
df_label = df_test_origin['Label']
data = df_test_origin.drop(columns=['Label'])
X_test_origin = data.values

# 5. Normalization

The values of the datasets are normalized using the Min-Max scaling technique, bringing them all within a range of [0,1].

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
scaler = MinMaxScaler()
X_test_n = scaler.fit_transform(Xtest)
X_test_n

array([[0.        , 1.        , 0.96969697, ..., 0.        , 0.71583333,
        0.71583333],
       [0.        , 1.        , 0.96969697, ..., 0.        , 0.6975    ,
        0.6975    ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 1.        , 0.44444444, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.28282828, ..., 0.        , 0.        ,
        0.        ]])

# 5. Standardization

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
scaler = StandardScaler()

X_test_sd = scaler.fit_transform(Xtest)

X_test_sd

array([[-0.33484556,  1.02685081,  1.44254915, ..., -0.13362864,
         2.21947035,  2.33346054],
       [-0.33484556,  1.02685081,  1.44254915, ..., -0.13362864,
         2.15086161,  2.26253235],
       [-0.33484556, -0.9738513 , -0.89916038, ..., -0.13362864,
        -0.45938921, -0.43596284],
       ...,
       [-0.33484556,  1.02685081,  0.17412315, ..., -0.13362864,
        -0.45938921, -0.43596284],
       [-0.33484556, -0.9738513 , -0.89916038, ..., -0.13362864,
        -0.45938921, -0.43596284],
       [-0.33484556, -0.9738513 , -0.21616177, ..., -0.13362864,
        -0.45938921, -0.43596284]])

In [20]:
for index, value in np.ndenumerate(X_test_sd):
    X_test_sd[index[0], 1] = X_test_n[index[0], 1]
 
X_test_sd

array([[-0.33484556,  1.        ,  1.44254915, ..., -0.13362864,
         2.21947035,  2.33346054],
       [-0.33484556,  1.        ,  1.44254915, ..., -0.13362864,
         2.15086161,  2.26253235],
       [-0.33484556,  0.        , -0.89916038, ..., -0.13362864,
        -0.45938921, -0.43596284],
       ...,
       [-0.33484556,  1.        ,  0.17412315, ..., -0.13362864,
        -0.45938921, -0.43596284],
       [-0.33484556,  0.        , -0.89916038, ..., -0.13362864,
        -0.45938921, -0.43596284],
       [-0.33484556,  0.        , -0.21616177, ..., -0.13362864,
        -0.45938921, -0.43596284]])

# 6. One-hot Encoding for labels

In [21]:
from tensorflow.keras.utils import to_categorical

In [22]:
y_test_origin = y_test

In [23]:
y_test = to_categorical(y_test, 15)

# 7.  Define the Metrics

In [24]:
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier

#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn.metrics import accuracy_score

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [25]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [26]:
labels_d = make_value2index(df_test['Label'])

In [27]:
print(labels_d)

{'BENIGN': 105018, 'Bot': 105298, 'DDoS': 124569, 'DoS GoldenEye': 126111, 'DoS Hulk': 160658, 'DoS Slowhttptest': 161486, 'DoS slowloris': 162320, 'FTP-Patator': 163498, 'Heartbleed': 163500, 'Infiltration': 163501, 'PortScan': 187347, 'SSH-Patator': 188173, 'Web Attack � Brute Force': 188382, 'Web Attack � Sql Injection': 188389, 'Web Attack � XSS': 188482}


## Convert each example as a 8x10 gray image since each example has 79 features by adding a new empty feature  

In [28]:
X_test_enlarge = np.append(X_test_sd, np.zeros([len(X_test_sd),1]),1)

In [29]:
img_row = 8
img_col = 10

X_test_gray = np.array([x.reshape(img_row, img_col,1) for x in X_test_enlarge])
test_gray = np.array([x.reshape(img_row, img_col) for x in X_test_enlarge])

X_test_gray.shape

(188483, 8, 10, 1)

# 8.  **Four Model Ensemble**

Here, all four models are reinstantiated and the best saved weights are loaded.

In [30]:
from tensorflow.keras.models import load_model
import pickle

### First, load the decision tree model without smote

In [31]:
# load the pre-trained model from google drive
decisiontree_file_name = '/content/drive/My Drive/CICIDS2017/decisiontree_smote_web.sav'

# load the pre-trained model locally
# decisiontree_file_name = 'decisiontree_smote_web.sav' 
decisiontree_model = pickle.load(open(decisiontree_file_name, 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [32]:
y_pred_tree = decisiontree_model.predict(X_test_origin)

In [33]:
display_metrics(y_test_origin,  y_pred_tree, labels_d)


Accuracy: 1.00

Micro Precision: 1.00
Micro Recall: 1.00
Micro F1-score: 1.00

Macro Precision: 0.90
Macro Recall: 0.97
Macro F1-score: 0.93

Weighted Precision: 1.00
Weighted Recall: 1.00
Weighted F1-score: 1.00

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       1.00      1.00      1.00    105019
                       Bot       0.98      0.97      0.97       280
                      DDoS       1.00      1.00      1.00     19271
             DoS GoldenEye       0.99      1.00      1.00      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       1.00      1.00      1.00       828
             DoS slowloris       0.99      0.99      0.99       834
               FTP-Patator       1.00      1.00      1.00      1178
                Heartbleed       0.67      1.00      0.80         2
              Infiltration       0.50      1.00      0.67         1
             

### Second, load the random forest model without smote

In [34]:
# load the pre-trained model from google drive
randomforest_file_name = '/content/drive/My Drive/CICIDS2017/randomforest_smote_web.sav'

# load the pre-trained model locally
# randomforest_file_name = 'randomforest_smote_web.sav'

randomforest_model = pickle.load(open(randomforest_file_name, 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [35]:
y_pred_rf = randomforest_model.predict(X_test_origin)

In [36]:
display_metrics(y_test_origin,  np.argmax(y_pred_rf, axis = 1), labels_d)


Accuracy: 1.00

Micro Precision: 1.00
Micro Recall: 1.00
Micro F1-score: 1.00

Macro Precision: 0.96
Macro Recall: 0.92
Macro F1-score: 0.93

Weighted Precision: 1.00
Weighted Recall: 1.00
Weighted F1-score: 1.00

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       1.00      1.00      1.00    105019
                       Bot       0.97      0.96      0.96       280
                      DDoS       1.00      1.00      1.00     19271
             DoS GoldenEye       1.00      1.00      1.00      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       1.00      0.99      1.00       828
             DoS slowloris       1.00      0.99      0.99       834
               FTP-Patator       1.00      1.00      1.00      1178
                Heartbleed       1.00      1.00      1.00         2
              Infiltration       1.00      1.00      1.00         1
             

### Third, load the DNN model

In [38]:
# load the pre-trained DNN from google drive
dnn_file_name = '/content/drive/My Drive/CICIDS2017/dnn_n_smote_http.h5'

# load the model locally
# dnn_file_name = 'dnn_n_smote_http.h5'

dnn_model = load_model(dnn_file_name, compile = False)

In [39]:
y_pred_dnn = dnn_model.predict(X_test_n)

In [40]:
display_metrics(y_test_origin,  np.argmax(y_pred_dnn, axis = 1), labels_d)


Accuracy: 0.99

Micro Precision: 0.99
Micro Recall: 0.99
Micro F1-score: 0.99

Macro Precision: 0.96
Macro Recall: 0.96
Macro F1-score: 0.96

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.99

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       0.99      1.00      0.99    105019
                       Bot       0.98      0.91      0.94       280
                      DDoS       1.00      1.00      1.00     19271
             DoS GoldenEye       1.00      0.99      1.00      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       0.99      1.00      1.00       828
             DoS slowloris       1.00      0.99      0.99       834
               FTP-Patator       1.00      1.00      1.00      1178
                Heartbleed       1.00      1.00      1.00         2
              Infiltration       1.00      1.00      1.00         1
             

### Finally, load the CNN2D model

In [41]:
# load the Random Forest model from disk

# load cnn2d model from google drive
cnn2d_file_name = "/content/drive/My Drive/CICIDS2017/cnn2d_http_smote"

# load local model
# cnn2d_file_name = 'cnn2d_78_sd_http80_without_normal.h5'
import tensorflow as tf
   
cnn2d_model =  tf.keras.models.load_model(cnn2d_file_name)

In [42]:
y_pred_cnn = cnn2d_model.predict(X_test_gray)

In [43]:
display_metrics(y_test_origin,  np.argmax(y_pred_cnn, axis = 1), labels_d)


Accuracy: 1.00

Micro Precision: 1.00
Micro Recall: 1.00
Micro F1-score: 1.00

Macro Precision: 0.96
Macro Recall: 0.97
Macro F1-score: 0.96

Weighted Precision: 1.00
Weighted Recall: 1.00
Weighted F1-score: 1.00

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       1.00      1.00      1.00    105019
                       Bot       0.97      0.97      0.97       280
                      DDoS       1.00      1.00      1.00     19271
             DoS GoldenEye       1.00      1.00      1.00      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       1.00      1.00      1.00       828
             DoS slowloris       1.00      1.00      1.00       834
               FTP-Patator       1.00      1.00      1.00      1178
                Heartbleed       1.00      1.00      1.00         2
              Infiltration       1.00      1.00      1.00         1
             

# 9. Ensemble results

Get the max values

In [44]:
y_pred_tree_cat = to_categorical(y_pred_tree, 15)

In [71]:
results = np.zeros((y_test_origin.shape[0], 15))
results = results+y_pred_rf*2 + y_pred_dnn*7 + y_pred_tree_cat*7+y_pred_cnn*5
display_metrics(y_test_origin, np.argmax(results, axis = 1) , labels_d)


Accuracy: 1.00

Micro Precision: 1.00
Micro Recall: 1.00
Micro F1-score: 1.00

Macro Precision: 0.99
Macro Recall: 0.98
Macro F1-score: 0.98

Weighted Precision: 1.00
Weighted Recall: 1.00
Weighted F1-score: 1.00

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       1.00      1.00      1.00    105019
                       Bot       0.98      0.97      0.97       280
                      DDoS       1.00      1.00      1.00     19271
             DoS GoldenEye       1.00      1.00      1.00      1542
                  DoS Hulk       1.00      1.00      1.00     34547
          DoS Slowhttptest       1.00      1.00      1.00       828
             DoS slowloris       1.00      1.00      1.00       834
               FTP-Patator       1.00      1.00      1.00      1178
                Heartbleed       1.00      1.00      1.00         2
              Infiltration       1.00      1.00      1.00         1
             

## Conclusion

 